# Bidding engine development

# Table of contents

- [Imports](#imports)
    - [Libraries](#libraries)
    - [Raw data](#raw-data)
- [Processing](#processing)
    - [Missing values](#missing-values)
        - [Forward fill](#filling-with-linear-ffill)
        - [Imputing with most frequent](#filling-with-most-frequent)
        - [Imputing with nearest neighbors](#filling-with-neighbours)
    - [Converting object features to numerical](#converting-object-features-to-numerical)
    - [Scaling metrics features](#scaling-metrics-features)
    - [Downcasting numerical features](#downcasting-numerical-features)
    - [Profiling interim data](#timeseries-of-the-random-feature-for-the-random-keyword)
- [Feature engineering](#feature-engineering)
    - [Add ROI](#add-roi)
    - [Convert CPC to GBP](#convert-cpc-to-gbp)
- [Quick look into keywords](#preliminary-keyword-analysis)
    - [Most sessions](#most-sessions)
    - [Least sessions](#least-sessions)
    - [Most impressions](#most-impressions)
    - [Least impressions](#least-impressions)
- [Tokenize](#tokenize)

# Imports
[top](#table-of-contents)

## Libraries


In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from sklearn.impute import KNNImputer, SimpleImputer
from transformers import AutoTokenizer

np.random.seed(1)
load_dotenv()

## Raw data

In [ ]:
raw_df = pd.read_feather(
    os.path.join(os.environ['RAW_DATA_PATH'], 'bidding_data.feather'))


In [ ]:
raw_df.info()

In [ ]:
raw_df.isna().sum()

In [ ]:
sum(raw_df.duplicated())

In [ ]:
raw_df.head()

Observations

1. 39 (out of 587,593) NaNs in `Cost` column. Nans rows can be dropped due to insignificant amount. Alternatively, can interpolated (forward fill)

2. The following columns could be converted to numerical (floats) judging by the `bidding_df.head()` and columns names
```
8   AbsoluteTopImpressionPercentage   587593 non-null  object        
9   TopImpressionPercentage           587593 non-null  object        
10  SearchImpressionShare             587593 non-null  object        
11  SearchTopImpressionShare          587593 non-null  object        
12  SearchRankLostTopImpressionShare  587593 non-null  object        
```

# Processing
[top](#table-of-contents)

In [ ]:
interim_df = raw_df.copy(deep=True)
interim_df_sorted = interim_df.sort_values(
    by=['CriterionId', 'Date']).reset_index(drop=True)


In [ ]:
def plot_keyword(df, criterion, features):
    """
    The plot_keyword function takes a dataframe, criterion ID, and list of features to plot.
    It then plots the specified features for the given criterion ID.
    
    Parameters
    ----------
        df
            Pass the dataframe to the function
        criterion
            Select the keyword to plot
        features
            Specify which columns to plot
    """
    for feature in features:
        plt.plot(df.loc[df['CriterionId'] == criterion, 'Date'],
                 df.loc[df['CriterionId'] == criterion, feature],
                 label=feature)
    plt.title('Keyword ID {}\nUnique keywords {}'.format(
        criterion, raw_df.loc[raw_df['CriterionId'] == criterion,
                              'Criteria'].unique()))
    plt.xticks(rotation=45)
    if len(features) == 1:
        plt.ylabel(feature)
    else:
        plt.legend()
    plt.show()

In [ ]:
def plot_filled(df, criterion, features):
    """
    The plot_filled function takes a dataframe, criterion ID, and list of features as input.
    It then plots the filled values for each feature in the list against the date.
    If there is only one feature in the list, it labels that axis with that feature name.
    
    Parameters
    ----------
        df
            Pass the dataframe to the function
        criterion
            Filter the dataframe to only show the rows that have a criterionid equal to criterion
        features
            Specify which columns to plot
    """
    from matplotlib.markers import MarkerStyle

    markers = list(MarkerStyle.markers.keys())[1:len(features) + 1]
    plt.figure()
    for feature, marker in zip(features, markers):
        if 'filled' in feature:
            plt.plot(df.loc[(df['CriterionId'] == criterion)
                            & (df['Cost'].isna()), 'Date'],
                     df.loc[(df['CriterionId'] == criterion) &
                            (df['Cost'].isna()), feature],
                     marker,
                     label=feature)
        else:
            plt.plot(df.loc[df['CriterionId'] == criterion, 'Date'],
                     df.loc[df['CriterionId'] == criterion, feature],
                     label=feature)
    plt.title('Keyword ID {}\nUnique keywords {}'.format(
        criterion,
        df.sort_values(
            by=['CriterionId', 'Date']).loc[df['CriterionId'] == criterion,
                                            'Criteria'].unique()))
    plt.xticks(rotation=45)
    if len(features) == 1:
        plt.ylabel(feature)
    else:
        plt.legend()
    plt.show()

## Missing values

In [ ]:
missing_criterionids = raw_df.loc[raw_df['Cost'].isnull(),
                                  'CriterionId'].unique()

In [ ]:
print(f'unique keywords with missing values: {len(missing_criterionids)}')

In [ ]:
criterion = np.random.choice(missing_criterionids)
features = ['Cost']
plot_keyword(df=interim_df_sorted, criterion=criterion, features=features)

### Filling with forward fill

In [ ]:
interim_df_sorted['Cost_forward_filled'] = interim_df_sorted[
    'Cost'].interpolate(method='ffill')


### Filling with most frequent

In [ ]:
frequency_imp = SimpleImputer(strategy='most_frequent')
interim_df_sorted['Cost_frequency_filled'] = interim_df_sorted['Cost']
for criterion in missing_criterionids:
    interim_df_sorted.loc[
        interim_df_sorted['CriterionId'] == criterion,
        'Cost_frequency_filled'] = frequency_imp.fit_transform(
            interim_df_sorted.loc[
                interim_df_sorted['CriterionId'] == criterion,
                'Cost_frequency_filled'].values.reshape(-1, 1)).reshape(-1)


### Filling with neighbours

In [ ]:
knn_imp = KNNImputer(n_neighbors=5, weights='distance')
interim_df_sorted['Cost_knn_filled'] = interim_df_sorted['Cost']
for criterion in missing_criterionids:
    interim_df_sorted.loc[
        interim_df_sorted['CriterionId'] == criterion,
        'Cost_knn_filled'] = knn_imp.fit_transform(interim_df_sorted.loc[
            interim_df_sorted['CriterionId'] == criterion,
            'Cost_knn_filled'].values.reshape(-1, 1)).reshape(-1)


Comparison of the filling methods on the random keyword

In [ ]:
criterion = np.random.choice(missing_criterionids)
plot_filled(df=interim_df_sorted,
            criterion=criterion,
            features=[
                'Cost', 'Cost_forward_filled', 'Cost_frequency_filled',
                'Cost_knn_filled'
            ])
print('keyword {}. mode: {}'.format(
    criterion,
    interim_df_sorted.loc[interim_df_sorted["CriterionId"] == criterion,
                          "Cost"].mode()[0]))


Conclusion on filling NaNs

Due to sparsity of data, filling with the most frequent per keyword value was chosen

In [ ]:
interim_df_sorted_no_nans = interim_df_sorted.copy(deep=True)
interim_df_sorted_no_nans['Cost'] = interim_df_sorted_no_nans[
    'Cost_frequency_filled']
interim_df_sorted_no_nans.drop(columns=[
    'Cost_forward_filled', 'Cost_frequency_filled', 'Cost_knn_filled'
],
                               inplace=True)
del knn_imp

## Converting object features to numerical

In [ ]:
object_columns = [
    'AbsoluteTopImpressionPercentage', 'TopImpressionPercentage',
    'SearchImpressionShare', 'SearchTopImpressionShare',
    'SearchRankLostTopImpressionShare'
]


In [ ]:
for col in object_columns:
    interim_df_sorted_no_nans[col] = interim_df_sorted_no_nans[col].str.lstrip(
        '<').str.rstrip('%').astype(float)


## Scaling metrics features

See the ranges of metrics in [Google Ad API](https://developers.google.com/google-ads/api/fields/v11/metrics)

In [ ]:
for col in object_columns:
    interim_df_sorted_no_nans[col] = (interim_df_sorted_no_nans[col] -
                                      interim_df_sorted_no_nans[col].min()
                                      ) / interim_df_sorted_no_nans[col].max()
    if 'Percentage' in col:
        interim_df_sorted_no_nans[col] *= 100

## Downcasting numerical features

In [ ]:
fcols = interim_df_sorted_no_nans.select_dtypes('float').columns
icols = interim_df_sorted_no_nans.select_dtypes('integer').columns

interim_df_sorted_no_nans[fcols] = interim_df_sorted_no_nans[fcols].apply(
    pd.to_numeric, downcast='float')
interim_df_sorted_no_nans[icols] = interim_df_sorted_no_nans[icols].apply(
    pd.to_numeric, downcast='integer')


## Profiling interim data

In [ ]:
if 'interim_data.html' not in os.listdir(os.environ['DATA_PROFILES_PATH']):
    from ydata_profiling import ProfileReport

    os.makedirs(os.environ['DATA_PROFILES_PATH'], exist_ok=True)
    interim_data_profile = ProfileReport(interim_df_sorted_no_nans,
                                         title='Interim Data Profile')
    interim_data_profile.to_file(
        os.path.join(os.environ['DATA_PROFILES_PATH'], 'interim_data.html'))


# Feature engineering
[top](#table-of-contents)

## Add ROI

In [ ]:
interim_df_sorted_no_nans['ROI_gbp'] = interim_df_sorted_no_nans[
    'Margin'].divide(interim_df_sorted_no_nans['Cost_gbp'],
                     fill_value=0).replace({
                         np.inf: 0,
                         np.nan: 0
                     })
interim_df_sorted_no_nans['ROI'] = interim_df_sorted_no_nans['Margin'].divide(
    interim_df_sorted_no_nans['Cost'], fill_value=0).replace({
        np.inf: 0,
        np.nan: 0
    })


## Convert CPC to GBP

In [ ]:
micros_to_gbp = interim_df_sorted_no_nans['Cost'].div(
    interim_df_sorted_no_nans['Cost_gbp']).replace({
        np.nan: 0,
        np.inf: 0
    }).unique()[1:].mean()


In [ ]:
interim_df_sorted_no_nans[
    'CpcBid_gbp'] = interim_df_sorted_no_nans['CpcBid'] / micros_to_gbp


In [ ]:
interim_df_sorted_no_nans['Criteria_list'] = interim_df_sorted_no_nans[
    'Criteria'].str.replace('+', '', regex=False).str.split()


In [ ]:
len(interim_df_sorted_no_nans.loc[(interim_df_sorted_no_nans['ROI_gbp'] > 2) &
                                  (interim_df_sorted_no_nans['Margin'] > 0) &
                                  (interim_df_sorted_no_nans['Sessions'] > 0),
                                  'CriterionId'].unique())


# Quick look into keywords
[top](#table-of-contents)

## Most sessions

In [ ]:
most_sessions = interim_df_sorted_no_nans.sort_values(
    by=['Sessions'],
    ascending=False).loc[interim_df_sorted_no_nans['ROI_gbp'] > 2,
                         'CriterionId'][:75].unique()
most_sessions

In [ ]:
interim_df_sorted_no_nans.loc[
    interim_df_sorted_no_nans['CriterionId'].isin(most_sessions),
    'Criteria'].unique()


## Least sessions

In [ ]:
least_sessions = interim_df_sorted_no_nans.sort_values(
    by=['Sessions'], ascending=True)['CriterionId'].unique()[:10]
least_sessions

In [ ]:
interim_df_sorted_no_nans.loc[
    interim_df_sorted_no_nans['CriterionId'].isin(least_sessions),
    'Criteria'].unique()


## Most impressions

In [ ]:
most_impressions = interim_df_sorted_no_nans.sort_values(
    by=['Impressions'], ascending=False)['CriterionId'].unique()[:10]
most_impressions

In [ ]:
interim_df_sorted_no_nans.loc[
    interim_df_sorted_no_nans['CriterionId'].isin(most_impressions),
    'Criteria'].unique()

## Least impressions

In [ ]:
least_impressions = interim_df_sorted_no_nans.sort_values(
    by=['Impressions'], ascending=True)['CriterionId'].unique()[:10]
least_impressions

In [ ]:
interim_df_sorted_no_nans.loc[
    interim_df_sorted_no_nans['CriterionId'].isin(least_impressions),
    'Criteria'].unique()


In [ ]:
interim_df = interim_df_sorted_no_nans
del interim_df_sorted_no_nans, interim_df_sorted, raw_df

# Tokenize
[top](#table-of-contents)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
interim_df['Criteria_tokenized'] = tokenizer.tokenize(
    interim_df['Criteria'].tolist(), padding=True, truncation=True)
